In [1]:
%config Completer.use_jedi = False
import warnings

warnings.filterwarnings(action='ignore')

# 앙상블 학습(Ensemble Learning)

지도 학습은 피쳐 데이터와 타겟 데이터를 이용해 전체 데이터를 분류하는 학습 방법이다.
지도 학습에 사용되는 데이터는 각 피쳐 데이터마다 타겟 변수가 올바르게 할당된 라벨링 데이터였고 이를 기반으로 학습 알고리즘을 생성하는 과정을 거쳤었다.

앙상블 학습의 핵심 아이디어는 트레이닝 데이터를 기반으로 분류 모형을 여러 개 만들고 서로 비교하는 것이다.
앙상블 학습 과정에서 만든 개별 분류 모형을 분류기(classifier)라고 하고 여러 개의 분류기를 결합함으로써 개별적인 분류기보다 성능이 뛰어난 최종 분류기를 만드는 것이 앙상블 학습의 목적이다.

# 보팅(Voting)
여러 개의 분류 모형의 결과를 대상으로 투표를 통해서 최종 클래스 레이블을 정하는 방법이다.

분류기가 10개 있다고 했을 때, 특정 데이터에 대해 7개의 분류기는 클래스 1이라고 예측하고, 나머지 3개의 분류 모형은 클래스가 2라고 예측했다고 했을 때 클래스 1이 가장 높은 득표수를 보이므로 최종적으로 클래스 1로 예측하는 것이다. 이를 다수결 투표(pluarlity voting)라고 하는데 이와 비슷한 방법으로 다수결이 아닌 절반 이상의 분류기의 표를 얻어야 하는 과반수 투표(majority voting) 방식이 있다.

개별 분류기는 지도 학습 방법 중 로지스틱 회귀, 서포트 벡터 머신, 의사 결정 트리 등 여러 가지 알고리즘 사용해서 다양한 분류 모형을 만들 수 있다.

보팅 알고리즘을 활용해 붓꽃 데이터를 분류하는 앙상블 모형을 만든다.

In [2]:
# 데이터 불러오기
from sklearn import datasets  # 붓꽃 데이터를 사용하기 위해 datasets를 import 한다.

raw_iris = datasets.load_iris()  # 붓꽃 데이터를 불러온다.

In [3]:
# 피처/타겟 데이터 지정
X = raw_iris.data  # 붓꽃 피쳐 데이터를 저장한다.
y = raw_iris.target  # 붗꽃 타겟 데이터를 저장한다.

In [4]:
# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split  # 트레이닝/테스트 데이터 분할을 위해 import 한다.

# train_test_split() 함수의 인수로 피쳐와 타겟 데이터를 넘겨 트레이닝 데이터와 테스트 데이터로 나눈다.
# 공부할 때는 매번 실행할 때 마다 결과가 달라지지 않도록 random_state를 지정하면 좋다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler  # 데이터 표준화를 위해 import 한다.

std_scale = StandardScaler()  # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)  # 트레이닝 피쳐 데이터 X_tn을 표준화 한다.
X_te_std = std_scale.transform(X_te)  # 테스트 피쳐 데이터 X_te를 표준화 한다.

In [6]:
# 보팅 학습
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀 알고리즘을 사용하기 위해 import 한다.
from sklearn.svm import SVC  # 서포트 벡터 머신 알고리즘을 사용하기 위해 import 한다.
from sklearn.naive_bayes import GaussianNB  # 가우시안 나이브 베이즈 알고리즘을 사용하기 위해 import 한다.
# 분류 문제이므로 VotingClassifier를 사용하지만 회귀 문제라면 VotingRegressor를 사용한다.
from sklearn.ensemble import VotingClassifier  # 보팅 알고리즘을 사용하기 위해 import 한다.

# 사이킷런의 LogisticRegression의 multi_class 옵션은 'ovr'일 경우 binary classification, 'multinomial'일 경우
# multi classification, 'auto'일 경우 타겟값 유형을 자동으로 해석해서 'ovr' 또는 'multinomial'을 적용한다.
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)  # 로지스틱 회귀 분석 객체를 만든다.
clf2 = SVC(kernel='linear', random_state=1)  # 서포트 벡터 머신 객체를 만든다.
clf3 = GaussianNB()  # 가우시안 나이브 베이즈 객체를 만든다.

# 앞서 만든 세 가지 모형을 이용해 보팅 모형을 만든다.
# estimators 옵션으로 앞서 만든 세 가지 모형을 지정한다.
# voting 옵션은 hard 또는 soft를 설정하는데 hard를 사용하면 투표 결과로 과반수가 넘는 라벨이 정해지고, soft로
# 설정하면 득표수가 가장 높은 라벨로 정해진다.
# weights 옵션으로 앞서 만든 세 가지 모형의 가중치를 지정한다.
clf_voting = VotingClassifier(estimators=[('lr', clf1), ('svm', clf2), ('gnb', clf3)], voting='hard',
                              weights=[1, 1, 1])
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 앙상블(보팅) 알고리즘을 학습시킨다.
clf_voting.fit(X_tn_std, y_tn)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 1])

In [7]:
# 데이터 예측
pred_voting = clf_voting.predict(X_te_std)  # 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_voting

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 2])

In [8]:
# 정확도 평가
from sklearn.metrics import accuracy_score  # 모형 정확도를 평가하기 위해 import 한다.

# accuracy_score() 함수의 인수로 실제 타겟 데이터와 예측된 데이터를 넘겨 정확도를 평가한다.
accuracy = accuracy_score(y_te, pred_voting)
accuracy

0.9736842105263158

In [9]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix  # 혼돈 행렬을 만들기 위해 import 한다.

# confusion_matrix() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_voting)
conf_matrix

array([[13,  0,  0],
       [ 0, 15,  1],
       [ 0,  0,  9]])

In [10]:
# 분류 리포트 확인
from sklearn.metrics import classification_report  # 분류 리포트를 출력하기 위해 import 한다.

# classification_report() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 분류 리포트를 만든다.
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

